In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd gdrive/

/content/gdrive


In [3]:
cd MyDrive/

/content/gdrive/MyDrive


In [4]:
cd END/s8

/content/gdrive/MyDrive/END/s8


In [7]:
ls

coqa-train-v1.0.json  LICENSE   train_light.json  tut3-model.pt
dev_light.json        qaps.csv  tut2-model.pt


In [8]:
import json
with open('coqa-train-v1.0.json') as f:
  input = json.load(f)

In [16]:
input["data"][0].keys()

dict_keys(['source', 'id', 'filename', 'story', 'questions', 'answers', 'name'])

In [26]:
questions,answers = [],[]

for i in range(len(input)):
  for j in range(len(input["data"][i]["questions"])):
    questions.append(input["data"][i]["questions"][j]["input_text"])
    for k in range(len(input["data"][i]["questions"])):
      if input["data"][i]["questions"][j]["turn_id"]==input["data"][i]["answers"][k]["turn_id"]:
        answers.append(input["data"][i]["answers"][k]["span_text"])


In [27]:
input["data"][0]["questions"]

[{'input_text': 'When was the Vat formally opened?', 'turn_id': 1},
 {'input_text': 'what is the library for?', 'turn_id': 2},
 {'input_text': 'for what subjects?', 'turn_id': 3},
 {'input_text': 'and?', 'turn_id': 4},
 {'input_text': 'what was started in 2014?', 'turn_id': 5},
 {'input_text': 'how do scholars divide the library?', 'turn_id': 6},
 {'input_text': 'how many?', 'turn_id': 7},
 {'input_text': 'what is the official name of the Vat?', 'turn_id': 8},
 {'input_text': 'where is it?', 'turn_id': 9},
 {'input_text': 'how many printed books does it contain?', 'turn_id': 10},
 {'input_text': 'when were the Secret Archives moved from the rest of the library?',
  'turn_id': 11},
 {'input_text': 'how many items are in this secret collection?',
  'turn_id': 12},
 {'input_text': 'Can anyone use this library?', 'turn_id': 13},
 {'input_text': 'what must be requested to view?', 'turn_id': 14},
 {'input_text': 'what must be requested in person or by mail?', 'turn_id': 15},
 {'input_text': 

In [21]:
input["data"][0]["answers"]

[{'input_text': 'It was formally established in 1475',
  'span_end': 179,
  'span_start': 151,
  'span_text': 'Formally established in 1475',
  'turn_id': 1},
 {'input_text': 'research',
  'span_end': 494,
  'span_start': 454,
  'span_text': 'he Vatican Library is a research library',
  'turn_id': 2},
 {'input_text': 'history, and law',
  'span_end': 511,
  'span_start': 457,
  'span_text': 'Vatican Library is a research library for history, law',
  'turn_id': 3},
 {'input_text': 'philosophy, science and theology',
  'span_end': 545,
  'span_start': 457,
  'span_text': 'Vatican Library is a research library for history, law, philosophy, science and theology',
  'turn_id': 4},
 {'input_text': 'a  project',
  'span_end': 879,
  'span_start': 769,
  'span_text': 'March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts',
  'turn_id': 5},
 {'input_text': 'into periods',
  'span_end': 1127,
  'span_start': 1048,
  'span_text': 'Scholars 

In [19]:
input.keys()

dict_keys(['version', 'data'])

In [31]:
answers

['Formally established in 1475',
 'he Vatican Library is a research library',
 'Vatican Library is a research library for history, law',
 'Vatican Library is a research library for history, law, philosophy, science and theology',
 'March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts',
 'Scholars have traditionally divided the history of the library into five period',
 'Scholars have traditionally divided the history of the library into five periods',
 'Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, ',
 'is the library of the Holy See, located in Vatican City.',
 ' It has 75,000 codices from throughout history, as well as 1.1 million printed books',
 'atican Secret Archives were separated from the library at the beginning of the 17th century;',
 ' Vatican Secret Archives were separated from the library at the beginning of the 17th century; they contain another 150,000 items. ',
 ' The V

In [30]:

len(answers),len(questions)

(25, 25)

In [32]:
# Import Library
import random
import torch, torchtext
from torchtext import data 

SRC = data.Field( tokenize = 'spacy',init_token = '<sos>', eos_token = '<eos>', lower = True)
TRG = data.Field( tokenize = 'spacy',init_token = '<sos>', eos_token = '<eos>', lower = True)
fields = [('src', SRC),('trg',TRG)]

In [34]:

# Manual Seed
SEED = 43
torch.manual_seed(SEED)
example = [data.Example.fromlist([questions[i],answers[i]], fields) for i in range(len(answers))]
ambignqlight = data.Dataset(example, fields)

In [35]:
train_data, test_data = ambignqlight.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [36]:
len(train_data),len(test_data)

(21, 4)

In [37]:
train_data,valid_data = train_data.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [38]:
len(train_data),len(valid_data)

(18, 3)

In [39]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [40]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [41]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time
BATCH_SIZE = 2

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,sort=False,
    device = device)

In [42]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [43]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
                
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

In [44]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

In [45]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)

In [46]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [47]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [48]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(25, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(54, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=54, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [49]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 6,550,326 trainable parameters


In [50]:
optimizer = optim.Adam(model.parameters())

In [51]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [52]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [53]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [54]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [55]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 0s
	Train Loss: 3.786 | Train PPL:  44.071
	 Val. Loss: 2.409 |  Val. PPL:  11.118
Epoch: 02 | Time: 0m 0s
	Train Loss: 3.330 | Train PPL:  27.950
	 Val. Loss: 2.217 |  Val. PPL:   9.182
Epoch: 03 | Time: 0m 0s
	Train Loss: 3.224 | Train PPL:  25.121
	 Val. Loss: 2.520 |  Val. PPL:  12.434
Epoch: 04 | Time: 0m 0s
	Train Loss: 3.167 | Train PPL:  23.734
	 Val. Loss: 2.193 |  Val. PPL:   8.965
Epoch: 05 | Time: 0m 0s
	Train Loss: 3.058 | Train PPL:  21.295
	 Val. Loss: 2.479 |  Val. PPL:  11.934
Epoch: 06 | Time: 0m 0s
	Train Loss: 2.900 | Train PPL:  18.176
	 Val. Loss: 2.214 |  Val. PPL:   9.153
Epoch: 07 | Time: 0m 0s
	Train Loss: 2.876 | Train PPL:  17.737
	 Val. Loss: 2.498 |  Val. PPL:  12.154
Epoch: 08 | Time: 0m 0s
	Train Loss: 2.846 | Train PPL:  17.212
	 Val. Loss: 2.418 |  Val. PPL:  11.221
Epoch: 09 | Time: 0m 0s
	Train Loss: 2.759 | Train PPL:  15.792
	 Val. Loss: 2.410 |  Val. PPL:  11.131
Epoch: 10 | Time: 0m 0s
	Train Loss: 2.704 | Train PPL:  14.944


In [56]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 2.377 | Test PPL:  10.778 |
